# keyword analysis

In [4]:
import numpy as np
from pecab import PeCab
from konlpy.tag import Mecab
# 워드클라우드 관련
from wordcloud import WordCloud
from PIL import Image
# 자연어처리 관련
import nltk
# 파일 처리 관련
import pandas as pd
import os
import re

In [5]:
# mecab 기준, 명사 & 형용사 & 동사 & 부사
FEATURE_POS = ['NNG', 'NNP', 'NNB', 'NNBC', 'NR',
               'NP', 'VV', 'VA', 'MAG']


def text_cleaning(doc):
    # 한국어를 제외한 글자를 제거하는 패턴.
    #doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)
    
    # 특수문자를 제거하는 패턴.
    doc = re.sub("[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]", " ", doc)
    
    # 영문 빼고 모두 제거하는 패턴.
    #doc = doc.replace("\n", " ")
    #doc = re.sub("[^A-Za-z ]", "", doc)
    
    return doc

def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    # SW.add("있다")
    SW.add("있어요")
    SW.add("대한")
    SW.add("합니다")
    SW.add("하는")
    
    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path, encoding="utf-8") as f:
        for word in f:
            SW.add(word.strip())
            
    return SW


def text_tokenizing(doc, tokenizer): 
    """
    Input Parameter :
    
    doc - tokenizing 하는 실제 데이터.
    tokenizer - token의 단위.
    """
    tok = PeCab() # 형태소 분석기 선언.
    
    if tokenizer == "words":
        return [word for word in doc.split() if word not in SW and len(word) > 1]
    
    elif tokenizer == "nouns":
        return [token for token in tok.nouns(doc)]
        
    elif tokenizer == "morphs":
        return [token for token in tok.morphs(doc)]
     
    elif tokenizer == "predefined":
        
        documents = []
        text_pos = [pair for pair in tok.pos(doc) if pair[0] not in SW and len(pair[0]) > 1]
        words = []

        for word, pos in text_pos:
            if pos in FEATURE_POS:
                words.append(word)

        return words

In [6]:
tokenizer = "predefined" # "words" / "nouns" / "morphs" / "predefined"

In [7]:
# stopwords : 불용어 --> token filtering.
SW = define_stopwords("stopwords-ko.txt")

In [8]:
document = pd.read_csv('./2023-05-11_21-14-14.csv', index_col=0)
documents = document['0']
tokenized_documents = documents.apply(text_cleaning)\
                               .apply(func=text_tokenizing, tokenizer=tokenizer)
tokenized_documents # 2d list

0     [공감, 댓글, 공유, 로그인, 소소, 달달, 여행, 일상, 이야기, 스윗, 수련,...
1     [공감, 댓글, 공유, 로그인, 블로그, 메뉴, 커피, 여행, 글쓰기, 에디터, 오...
2     [공감, 댓글, 공유, 로그인, 블로그, 메뉴, 글쓰기, 에디터, 오픈, 그래픽, ...
3     [공감, 댓글, 공유, 로그인, 소소, 꾸준히, 블로그, 메뉴, 글쓰기, 에디터, ...
4     [공감, 공유, 로그인, 생각, 블로그, 메뉴, 글쓰기, 에디터, 오픈, 챌린지, ...
                            ...                        
65    [공감, 댓글, 공유, 로그인, 소소, 꾸준히, 블로그, 메뉴, 글쓰기, 에디터, ...
66    [공감, 댓글, 공유, 로그인, 스피릿, 블로그, 블로그, 메뉴, 글쓰기, 에디터,...
67    [공감, 댓글, 공유, 로그인, 소소, 꾸준히, 블로그, 메뉴, 글쓰기, 에디터, ...
68    [공감, 댓글, 공유, 로그인, 소소, 꾸준히, 블로그, 메뉴, 글쓰기, 에디터, ...
69    [공감, 댓글, 공유, 로그인, 소소, 꾸준히, 블로그, 메뉴, 글쓰기, 에디터, ...
Name: 0, Length: 70, dtype: object

In [13]:
tokenized_documents.to_csv("tokenized_documents.csv", encoding="utf-8-sig")

# word cloud